# Pushing your Kitura application to IBM Cloud

Now that you've worked on the main logic behind the 2048-playing AI, it's time to expose it to the web! To do so, you'll be using a slightly modified version of Gabriele Cirulli's original 2048 game from Github.

Here's the architecture of the solution:

1. You'll define the Monte Carlo Tree Search function as in Lab 4.
1. You'll use Kitura to expose a REST API which takes a 2048 game state, and returns the best move to take.
1. You'll use a pre-modified version of the 2048 game which works the same, except that it listens for a keypress on "t". When you click the "t" key, it'll start autoplaying the game by using the REST API you create.
1. Kitura will host the 2048 game itself, enabling you to interact with it on IBM Cloud.

When you're pushing a Swift application to IBM Cloud, there's a lot of "boilerplate" code that must be repeated each time. In some rare cases, you may wish to modify this code for some exotic functionality. However, in this case, we'll be keeping that as-it-is. So, in order to reduce the amount of code you need to write, I've included the majority of it in a Github repo for you already. Let's clone that here:

In [ ]:
%%bash
git clone https://github.com/tanmayb123/TB2048

This Github repo has the following directory structure:

```
-Tanmay2048
 |-Sources (The source files for the program)
   |-NN2048 (The name of the build target)
     |-2048.swift (The file containing the 2048 logic you built in lab 3)
     |-main.swift (The file containing the Kitura logic you'll learn about here)
 |-public (The source files for the 2048 Web Interface by Gabriele Cirulli)
   |- ...
 |-Package.swift (The file telling Swift to download and link Kitura)
 |-Procfile (The file telling IBM Cloud how to run your app)
```

The code in `public` is [taken from here](https://github.com/gabrielecirulli/2048). It's the original implementation of the 2048 game. However, one modification has been made: the key press listener now also listens for keystrokes of the "t" key. When you press "t", it'll activate the self-playing loop.

The `Sources/NN2048/2048.swift` file is the file you created in Lab 3 - no difference whatsoever.

The `Package.swift` file contains the following content:

```
// swift-tools-version:4.0
// The swift-tools-version declares the minimum version of Swift required to build this package.

import PackageDescription

let package = Package(
    name: "NN2048",
    products: [
      .executable(
        name: "NN2048",
        targets:  ["NN2048"]
      )
    ],
    dependencies: [
      .package(url: "https://github.com/IBM-Swift/Kitura.git", from: "2.6.0"),
    ],
    targets: [
      .target(
        name: "NN2048",
        dependencies: ["Kitura"]
      )
    ]
)
```

The idea behind this file is very simple - it tells swift the following things:
1. There's a build target called "NN2048", and it:
    1. Builds to an executable.
    1. Requires version 2.6 of `Kitura`.
    
Then, the `Procfile` makes sure that IBM Cloud knows how to build and run your application. This is the content:

`web: swift run -Xswiftc -O -Xcxx -Ofast -Xcxx -funroll-loops -Xcc -ffast-math -Xcc -Ofast`

That's right - just one line. Essentially, all these compiler flags are _kind of_ unnecessary. However, without them, your program would be debilitatingly slow. These flags tell Swift to:
1. Compile Swift with full speed optimizations.
1. Tell the C & C++ compilers to use full speed optimizations.
1. Tell the C++ compiler to unroll loops (this is an optimization that makes compile time slower, but run time faster).

And, finally, the cherry on top, `main.swift`! This is what the file contains:

```
import Foundation
import Kitura

let router = Router()
router.all("/", middleware: StaticFileServer())

router.get("/analyze") { request, response, next in
    let boardReceived = request.queryParameters["board"]!.components(separatedBy: ",").map({ Int($0)! })
    let shapedBoard = (0..<4).map({ row in (0..<4).map({ col in boardReceived[4 * col + row] }) })
    let game = Game2048(board: shapedBoard)
    response.send(monteCarloSearch(state: game).string)
    next()
}

Kitura.addHTTPServer(onPort: 8080, with: router)
Kitura.run()
```

I'm nearly sure you should be able to understand this code by now. Let me take you through it step by step:
1. You start by importing Foundation and Kitura.
1. You tell Kitura that you need a `StaticFileServer` on endpoint `/`. This means that whenever someone goes to the root of this website, they'll see the contents from the `public` folder. So, for example, if your app name is `tb2048`, when someone heads over to `tb2048.mybluemix.net/`, they'll be greeted with the `index.html` page from the `public` folder you created. This is possible thanks to the `StaticFileServer`.
1. You tell Kitura to take any GET requests sent to the `/analyze` endpoint, and call a function to figure out what to do next. This function that we've defined will take the board state that was passed through the REST API, and create a convenient internal representation out of it (you built this in lab 3). Then, it'll use Monte Carlo Tree Search to find the best move and return it as a response.
1. You finally tell Kitura to start this HTTP server on port 8080, which is the port that IBM Cloud expects you to run on.

It's _that_ easy to use Kitura! Now, let's go ahead and push this to IBM Cloud. To do so, you'll need to log in first. In the cell below, insert the following in sequence:

```
[email_address]
[password]
[region]
```

So, for example, it could look like:

```
%%bash
ibmcloud login
john.appleseed@email.com
mysecurepassword
5
```

If you're unaware of your region number, use the following as a reference:

```
1. au-syd
2. jp-tok
3. eu-de
4. eu-gb
5. us-south
6. us-east
```

Once you've filled it out, run the cell.

In [ ]:
%%bash
ibmcloud login


You should be logged in to IBM Cloud. Now, let's deploy! Simply fill out the following fields in the following cell:

1. `[email]` - this is the email address associated with your IBM ID.
1. `[appname]` - this is the name that you chose while creating your app.

Run the cell, and wait for it to execute.

In [ ]:
%%bash
cd TB2048
ibmcloud target -o [email] -s dev
ibmcloud app push [appname]

And ... there you go! That's all there is to it! Now head over to the endpoint you chose when you created your application. You should be able to play 2048 like normal. However, right as you press `t`, the game should start playing itself!

**NOTE: you usually should share your notebooks - but this isn't one of them. This notebook contains sensitive information like your password. Be careful!**